# Convert HyperNeRF Data to EPIC-Diff Style

In [15]:
import numpy as np
from pathlib import Path
import json
import imageio
import shutil

epic_dir = Path('data/EPIC-Diff/')
hypernerf_dir = Path('/home/tw554/Neural-Scene-Flow-Fields/data/')
# hypernerf_dir = Path('data/hypernerf/')
scene_id = 'kubric_single_car_rand_v2'
epic_data_dir = epic_dir / scene_id
epic_rgb_dir = epic_data_dir / 'frames'
epic_mask_dir = epic_data_dir / 'annotations'

epic_rgb_dir.mkdir(exist_ok=True, parents=True)
epic_mask_dir.mkdir(exist_ok=True, parents=True)

root_dir = hypernerf_dir / scene_id
img_scale = 2
# Where to save RGB images.
rgb_dir = root_dir / 'rgb' / f'{img_scale}x'
# Where to save mask images.
mask_dir = root_dir / 'mask' / f'{img_scale}x'
camera_dir = root_dir / 'camera-gt'
if not camera_dir.exists():
    camera_dir = root_dir / 'camera'

cam_list = sorted(list(camera_dir.glob('*')))

with open(root_dir / 'dataset.json','r') as f:
    dataset_data = json.load(f)
with open(root_dir / 'metadata.json','r') as f:
    time_data = json.load(f)

if (root_dir / 'scene_gt.json').exists():
    with open(root_dir / 'scene_gt.json','r') as f:
        scene_data = json.load(f)
else:
    with open(root_dir / 'scene.json','r') as f:
        scene_data = json.load(f)

rgb_list = [rgb_dir / f'{idx}.png' for idx in dataset_data['ids']]
if mask_dir.exists():
    mask_list = [mask_dir / f'{idx}.png' for idx in dataset_data['ids']]
else:
    mask_list = []

# mask_list = sorted(list(mask_dir.glob('*.png')))
# rgb_list = sorted(list(rgb_dir.glob('*.png')))

In [16]:
# copy all the images and masks
for mask in mask_list:
    # shutil.copy(str(mask), str(mask_dir))
    img = imageio.imread(str(mask))
    imageio.imsave(str(epic_mask_dir / f'{mask.stem}.bmp'), img)
    

for rgb in rgb_list:
    # shutil.copy(str(rgb), str(rgb_dir))
    img = imageio.imread(str(rgb))
    imageio.imsave(str(epic_rgb_dir / f'{rgb.stem}.bmp'), img)

image_size = [img.shape[1], img.shape[0]]


In [17]:
# write meta
meta = {}

meta['ids_all'] = list(range(len(rgb_list)))
meta['ids_train'] = [dataset_data['ids'].index(idx) for idx in dataset_data['train_ids']]
# tes and val not used atm
meta['ids_val'] = [dataset_data['ids'].index(idx) for idx in dataset_data['val_ids']]
meta['ids_test'] = meta['ids_all']
# meta['ids_val'] = meta['ids_train']
# meta['ids_test'] = meta['ids_train']

with open(cam_list[0],'r') as f:
        cam_data = json.load(f)
intrinsics = np.zeros([3,3])
intrinsics[2,2] = 1
intrinsics[0,0] = intrinsics[1,1] = cam_data['focal_length'] / img_scale
intrinsics[0,2] = cam_data['principal_point'][0] / img_scale
intrinsics[1,2] = cam_data['principal_point'][1] / img_scale
meta['intrinsics'] = intrinsics.tolist()

meta['images'] = {}
meta['nears'] = {}
meta['fars'] = {}
meta['poses'] = {}

for i in range(len(rgb_list)):
    meta['images'][str(i)] = f'{rgb_list[i].stem}.bmp'
    meta['nears'][str(i)] = scene_data['near'] / scene_data['scale']
    meta['fars'][str(i)] = scene_data['far'] / scene_data['scale']

    rgb_list[i].stem

    with open(camera_dir / f'{rgb_list[i].stem}.json','r') as f:
        cam_data = json.load(f)

    c2w = np.zeros([3,4])
    c2w[:3,:3] = np.array(cam_data['orientation']).T
    c2w[:3,3] = cam_data['position']
    meta['poses'][str(i)] = c2w.tolist()

with open(epic_data_dir / 'meta.json', 'w') as f:
    json.dump(meta, f, indent=2)

print(f"Num of imgs: {len(meta['ids_all'])}")
print(f"Num of train: {len(meta['ids_train'])}")
print(f"Num of val: {len(meta['ids_val'])}")
print(f"Num of test: {len(meta['ids_test'])}")



Num of imgs: 200
Num of train: 200
Num of val: 0
Num of test: 200


In [18]:
intrinsics

array([[280.,   0., 128.],
       [  0., 280., 128.],
       [  0.,   0.,   1.]])